# Quantum enhanced support vector machine


A support vector machine is a supervised machine learning method that is trained using a dataset and will predict if a particular observation is in a certain class based upon what it has been trained on. It is similar to a linear classifier in that it uses a hyperplane to separate classes.

Quantum Support Vector Machine (QSVM) is a quantum version of the Support Vector Machine (SVM) algorithm which uses quantum laws to perform calculations. QSVM uses the power of Quantum Computer/technology to improve the performance of classical SVM algorithms that run on classical machines with CPUs or GPUs.

**Quantum Machine Learning takes on three major steps;** 

- 1st we will have to translate the classical data point $\vec{x}$ into a quantum datapoint $\vert \Phi{(\vec{x})} \rangle$. This can be achieved by a circuit $\mathcal{U}_{\Phi(\vec{x})} \vert 0 \rangle $. Where $\Phi()$ could be any classical function applied on the classical data $\vec{x}$.

- 2nd we need a parameterized quantum circuit $W( \theta )$ that processes the data in a way that in the end we...

- 3rd can apply a measurement that returns a classical value $-1$ or $1$ for each classical input $\vec{x}$ that indentifies the label of the classical data.

**The most general ansatz**

Following these steps we can define an ansatz for this kind of problem which is $W(\theta) \mathcal{U}_{\Phi}(\vec{x}) \vert 0 \rangle$.

These kind of ansatz are called quantum variational circuits.

In [6]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.components.multiclass_extensions import (ErrorCorrectingCode,AllPairs,OneAgainstRest)
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import get_feature_dimension
from qiskit import IBMQ


# IMPLEMENTATION


The bookend refers whether we want to simulate the experiment in classical device
or run in real quantum device.

Quantum Devices

• IBM Q 14 Melbourne: The IBM Q 14 Melbourne is a 14 qubit quantum
computer developed by IBM.

• IBM Q QASM Simulator: IBM has developed this IBM Q QASM simulator that can simulate
32qubits operation. This simulator can take the algorithms and run them on
IBM Q systems through the IBM Q Experience or the IBM Q Network without
any code changes.

We used IBM Q QASM Simulator.

In [7]:
from qiskit import Aer  # simulator framework from qiskit

# will create a statevector of possibilities. 
sim_backend = Aer.get_backend('qasm_simulator')
#Qiskit Aer: accelerates the development via simulators, emulators, and debuggers

The easiest way to implement the dataset in through numpy arrays:

In [8]:
training_data = {'A': np.asarray([[0.324],[0.565]]),'B': np.asarray([[1.324],[1.565]])}

Where A is the numpy array for smokers and B is the array for Non-smokers. Next we implement a testing dataset.

In [9]:
testing_data = {'A': np.asarray([[0.024],[0.456]]),'B': np.asarray([[1.777],[1.341]])}

Next we specify the number of qubits to be used. Rule of thumb is N qubits for N features. Since we are using 1 feature we are using 1 qubit.

# Build QISKIT circuit for feature map

In [10]:
num_qubits = 1

Then we specify the feature map. This maps the data with second order expansion by entangling qubits.

In [11]:
feature_map = SecondOrderExpansion(feature_dimension=num_qubits,depth=2,entanglement='full')

The function `SecondOrderExpansion` has the arguments `feature_dimension`, which is the dimension of the input data $\vec{x}$ and at the same time also the number of qubits. `depth` is the number of repetitions of the feature map.This is normally defaulted to 2. entanglement = ‘full’ means all qubits will be entangled with each other.

The form of the map is a dictionary; each entry in the dictionary has a source qubit
index as the key, with the corresponding value being a list of target qubit indexes
to which the source qubit should be entangled .

**Feature map**
The quantum feature map of depth $d$ is implemented by the unitary operator;

$\mathcal{U}_{\Phi(x)} = U_{\Phi(x)} \otimes H^{\otimes n}$, where $U_{\Phi(x)} = \exp \left( i \sum_{S \in n} \phi_S(x) \prod_{i \in S} Z_i \right)$, which simplifies a lot when we (like in [1](https://arxiv.org/pdf/1804.11326.pdf)) only consider $S \leq 2$ interactions, which means we only let two qubits interact at a time.

which contains layers of Hadamard gates interleaved with entangling blocks encoding the classical data as shown in circuit diagram below for $d=2$.

The depth $d=1$ version of this quantum circuit is shown in the figure below for $n=2$ qubits.
![](second.png)


detailed information can be found in [1](https://arxiv.org/pdf/1804.11326.pdf)

### QSVM Algorithm

Qiskit aqua also provides a pre-defined function to train the whole QSVM. Where we only have to provide the feature map, a training and a test set and Qiskit will do all the work for us.

Apart from finding the quantum Kernel the QSVM algorithm does only classical optimization. In the end there is no difference to the classical SVM, except that the Kernels are coming from a quantum distribution.

QSVM will minimize the loss 
$$L(W) = \sum_i \alpha_i - \frac{1}{2}\sum_{i,j} y_i y_j \alpha_i \alpha_j K(\vec{x}_i, \vec{x}_j)$$
via optimizing the parameters $\vec{\alpha}$.

After training we can predict a label $y'$ of a data instance $\vec{s}$ with $y' = \text{sign}\left( \sum_i y_i \alpha_i K(\vec{x}_i, \vec{s}) \right)$.

In [12]:
qsvm = QSVM(feature_map, training_data,testing_data) # Creation of QSVM

#### Run QSVM

Finally we will have to define where we would like to run this algorithm. For now we will run it on a local QASM Simulator. But the algorithm could also be sent to the IBMQ an be evaluated on a real quantum computer.

We will have to define the `shots`, which are the number of measurements that we will take for each qubit. And for better reproducability we also set here the random seeds `seed_simulator` and `see_transpiler`.

In [13]:
from qiskit.aqua import run_algorithm, QuantumInstance
from qiskit import BasicAer

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=10598, seed_transpiler=10598)





Next we run the QSVM to get the accuracy using the following code. This is where it is sent as a job to the quantum device.

In [14]:
result = qsvm.run(quantum_instance)

In [15]:
data = np.array([[1.453],[1.023],[0.135],[0.266]]) #Unlabelled data

In [16]:
prediction = qsvm.predict(data,quantum_instance) # Predict using unlabelled data 

print('Prediction of Smoker or Non-Smoker based upon gene expression of CDKN2A\n')
print('Accuracy: ' , result['testing_accuracy'],'\n')
print('Prediction from input data where 0 = Non-Smoker and 1 = Smoker\n')
print(prediction)

Prediction of Smoker or Non-Smoker based upon gene expression of CDKN2A

Accuracy:  1.0 

Prediction from input data where 0 = Non-Smoker and 1 = Smoker

[1 1 0 0]


# References

[1] Vojtech Havlicek, Antonio D. C´orcoles, Kristan Temme, Aram W. Harrow, Abhinav Kandala, Jerry M. Chow, and Jay M. Gambetta1, *Supervised learning with quantum enhanced feature spaces*, Nature 567, 209–212 (2019).

[2] Pattern Recognition with Quantum Support Vector Machine(QSVM) on Near Term Quantum Processors.

[3] https://quantumcomputinguk.org/tutorials/how-to-implement-qsvm-algorithm-on-ibms-quantum-computers-with-qiskit.

[4] https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/machine_learning/custom_feature_map.ipynb